# Entraînement d'un CNN avec des images de spectrogrammes

Ce notebook entraîne un réseau de neurones convolutionnel (CNN) en utilisant des images de spectrogrammes des vibrations du moteur. Le jeu de données est divisé en ensembles d'entraînement, de validation et de test. Le modèle entraîné est sauvegardé avec un horodatage et une convention de nommage basée sur les performances.

## Importation des bibliothèques (libraries) nécessaires

Nous allons importer les bibliothèques nécessaires pour le chargement des données, le prétraitement et la construction du modèle CNN.

In [ ]:
!pip install tensorflow keras matplotlib numpy pandas scikit-learn pillow

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import img_to_array, load_img
from datetime import datetime

## Chargement et prétraitement des données

Nous allons charger les images de spectrogrammes depuis le répertoire `data/05_cnn_input`, les prétraiter, puis les diviser en ensembles d'entraînement, de validation et de test.

In [ ]:
# data_dir = "./../../data/05_cnn_input"
# data_dir = "/tf/data/05_cnn_input" 

# Dedecter si on est sous Windows ou Linux (Linux = Docker )
if os.name == 'nt':
    print("On est sous Windows.")
    data_dir = "./../../data/05_cnn_input"
else:  # Ubuntu
    print("On est sous Linux.")
    data_dir = "/tf/data/05_cnn_input"



print("Contenu du dossier 05_cnn_input :", os.listdir(data_dir))

image_paths = []
labels = []

for label, category in enumerate(['balourd', 'sain']):
    category_dir = os.path.join(data_dir, category)
    if not os.path.exists(category_dir):
        print(f"Directory does not exist: {category_dir}")
        continue
    print(f"Checking directory: {category_dir}")
    for root, _, files in os.walk(category_dir):
        print(f"Found {len(files)} files in {root}")
        for file in files:
            if file.endswith(".png"):
                image_paths.append(os.path.join(root, file))
                labels.append(label)

print(f"Total directories checked: {len(['balourd', 'sain'])}")
print(f"Total image paths collected: {len(image_paths)}")
print("Sample image paths:", image_paths[:5])
print("Labels distribution:", {label: labels.count(label) for label in set(labels)})

image_paths = np.array(image_paths)
labels = np.array(labels)

In [ ]:

# Prétraitement des images : charger les images telles quelles (sans redimensionnement ni rognage)
def preprocess_image(image_path):
    try:
        img = load_img(image_path)  # Charger l'image sans redimensionnement
        img_array = img_to_array(img)
        return img_array
    except Exception as e:
        print(f"Erreur lors du traitement de l'image {image_path} : {e}")
        return None

# Débogage : Afficher les informations sur le jeu de données
print(f"Nombre total de chemins d'images trouvés : {len(image_paths)}")
print("Exemples de chemins d'images :", image_paths[:5])
print("Répartition des étiquettes :", {label: labels.tolist().count(label) for label in set(labels)})

# Appliquer le prétraitement à toutes les images
images = np.array([img for img in (preprocess_image(path) for path in image_paths) if img is not None])

# Débogage : Vérifier les images traitées
print(f"Nombre total d'images valides traitées : {len(images)}")
if len(images) > 0:
    print("Dimensions de la première image :", images[0].shape)

# Diviser le jeu de données en ensembles d'entraînement, de validation et de test
if len(images) == 0:
    raise ValueError("Aucune image valide n'a été traitée. Veuillez vérifier le jeu de données.")

X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Débogage : Afficher les répartitions des ensembles de données
print(f"Ensemble d'entraînement : {len(X_train)} échantillons")
print(f"Ensemble de validation : {len(X_val)} échantillons")
print(f"Ensemble de test : {len(X_test)} échantillons")

## Construction du modèle CNN

Nous allons définir une architecture de réseau de neurones convolutionnel (CNN) pour traiter les images de spectrogrammes.

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from datetime import datetime

# Generate a timestamp for the filename
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Define Model Checkpoint
model_checkpoint = ModelCheckpoint(
    filepath=f'best_model_{timestamp}.h5',
    monitor='val_accuracy', 
    save_best_only=True,
    mode='max',
    verbose=1
)


# Define Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)


# Définir le modèle CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu',input_shape=(129, 85, 3)),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    # Conv2D(128, (3, 3), activation='relu'),
    # BatchNormalization(),
    # MaxPooling2D((2, 2)),

    # Conv2D(256, (3, 3), activation='relu'),
    # BatchNormalization(),
    # MaxPooling2D((2, 2)),

    Dropout(0.2),


    Flatten(),
    
    
    # Dense(256, activation='relu'),
    # Dropout(0.5),

    # Dense(128, activation='relu'),
    # Dropout(0.5),

    Dense(32, activation='relu'),
    Dropout(0.2),

    Dense(1, activation='sigmoid')
])

# Afficher le résumé du modèle
model.summary()

## Entraînement du modèle et sauvegarde

Nous allons compiler le modèle, l'entraîner en utilisant les ensembles d'entraînement et de validation, puis enregistrer le modèle entraîné avec une convention de nommage basée sur les performances.

In [ ]:

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint] 
)
# Générer un nom de fichier avec un timestamp et les performances
def generate_model_name(history, test_accuracy):
    val_accuracy = max(history.history['val_accuracy']) * 100
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    return f"{timestamp}_validation_res_{val_accuracy:.2f}_test_set_{test_accuracy:.2f}.h5"

## Évaluation du modèle

Nous allons évaluer le modèle entraîné sur l'ensemble de test et calculer la précision sur cet ensemble.

In [ ]:
## Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble de test : {test_accuracy * 100:.2f}%")

if test_accuracy * 100 > 97:
    # Sauvegarder le modèle avec la précision mise à jour
    model.save(generate_model_name(history, test_accuracy * 100))
    print("Modèle sauvegardé avec succès.")
else:
    print("La précision sur l'ensemble de test est inférieure à 98%. Le modèle ne sera pas sauvegardé.")

# # Visualiser quelques prédictions
# import matplotlib.pyplot as plt
# predictions = (model.predict(X_test) > 0.5).astype("int32")
# plt.figure(figsize=(12, 12))
# for i in range(16):
#     plt.subplot(4, 4, i + 1)
#     plt.imshow(X_test[i].astype("uint8"))
#     plt.title(f"Label: {'balourd' if y_test[i] == 0 else 'sain'}\nPred: {'balourd' if predictions[i] == 0 else 'sain'}")
#     plt.axis('off')
# plt.tight_layout()
# plt.show()